In [1]:
from langchain_community.chat_models import ChatOpenAI
import dotenv
import os
from discussion_agents.cog.agent.reflexion import ReflexionReActAgent

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai_api_key)


Given we change the examples/prompts for agent/reflect, what changes must be made?
- the states of the agent

The states of the reflexion react agent are: reflector and memory
- memory stores scratchpad info only
- reflector stores reflections only

Given the above context, if we modify the examples, prompt, reflect_examples, and reflect_prompt, what happens to the agent states?
- if we modify both/either examples or prompt, the input to the llm for prompting the agent is diff but
the outputs won't differ; memory stores scratchpad (output) only so no change here; reflector does not change in this case
- if we modify both/either examples or prompt, the input to the llm reflect is diff but 
the outputs won't differ; reflector stores reflections (output) only so no change here; memory does not store reflections

So memory and reflection don't bleed into each other. They are mutually exclusive? Yes.

If I make changes in the prompt for the agent (examples/prompt), should these changes be reflected in the prompt for reflection?
For example, if I include "insights" or fewshot examples in the prompt for the agent, shouldn't these also be present in the 
prompt input during reflection?

Reflection takes in "question", "examples", "reflections". In this case, examples refers to fewshot reflection examples not the ones used 
in the prompting for the agent. I notice in original react and reflexion, they don't include these fewshot examples in reflection and it
wouldn't make much sense to anyways. It's not relevant context for the sake of reflection.

What about insights? That's slightly different from reflections, shouldn't that be included in the prompts for reflection (since it's
used for prompting the agent). This does make slight sense, except these insights are meant to be fixed and not updated (unless it's done by
the expel insight extraction stage 2 process) by the llm during reflection. So if it were to be included during reflexion, it would be static/fixed.

Ok, but then if it's static/fixed in the reflection prompt, don't you think it'll help out the reflection process? 

The ExpeL paper uses basic ReAct during stage 3 evaluation. I have no idea how they would go about it with reflexion + react. That being said,
generally, nothing in the input to the prompt agent is used as input to the reflect component of reflexion. This is true for CoT and react.

Hmmmmmm, then what's stopping you from incorporating these insights into the prompt? Well, first off, we know that these insights shouldn't be 
part of the reflection output/reflector class state. But now it begs the question: should it be part of the input to the reflection component?

My answer is a bit mixed on this. It makes sense to include it, but generally you don't include input to prompt agent as the input to the reflection process.

It kinda makes sense that the insights would aid in the reflection process, but then how would this even look? Well, the reflect prompt
would probably have either a new argument (unlikely) or the insights are appended to the examples (probably the case). The problem is
the reflector should be focused on the reflection process. These insights aid in inference not reflection. That's what they're geared for.

I'm leaning towards no. if it is yes, then it would have to be appended to the examples or we could have kwargs (but this gets complicated
very quickly; let's stay away from this).

So if we do do it, it must be through appending the examples. Though I don't think this matters because we won't do this. It wouldn't make
sense. The insights are for inference not for reflection, but then again, the insights would be partially responsible for inference output.

Ok, I think I won't do it. But, to be comprehensive, is there harm in providing the option to do that? No. Let's just implement something and see where this takes us. oK


In [2]:
import joblib 
from discussion_agents.cog.modules.memory.expel import ExpeLExperienceMemory

hotpotqa = joblib.load('../../../tests/assets/hotpotqa/hotpot-qa-distractor-sample.joblib')
experiences = joblib.load('../../../tests/assets/expel/expel_experiences_10_fake.joblib')
memory = ExpeLExperienceMemory(experiences)

c:\Users\tuvin\anaconda3\envs\discussion-agents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
queries = {
    "task": 'The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? ',
    "thought": 'Thought: I should try a different approach. Let me search for press releases, industry news sources, or announcements specifically related to the name change and new acronym for VIVA Media AG in 2004. By focusing on more specialized sources, I may be able to find the accurate information needed to answer the question correctly. ',
    "other": "Some other query."
}

memory_dict = memory.load_memories(query=queries["task"])['fewshots']

In [4]:
print("\n\n".join(memory_dict + ["(END OF EXAMPLES)"]))

The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? 
Thought: I need to search for the creator of "Wallace and Gromit" and find out what other animation comedy they created that involves animated zoo animals and people talking about their homes. 
Action: Action 1: Search[creator of Wallace and Gromit]
Observation 1: Invalid Action. Valid Actions are Lookup[<topic>] Search[<topic>] and Finish[<answer>].
Thought: I should search for the creator of "Wallace and Gromit" by searching for Nick Park. 
Action: Search[Nick Park]
Observation 2: Could not find [Nick Park]. Similar: ['Nick Park', 'Wallace and Gromit', 'Nick Hornby', 'Aardman Animations', 'Chicken Run', 'A Grand Day Out', 'Harry Gregson-Williams', 'Plasticine', 'Nickelodeon', 'Wallace & Gromit: The Curse of the Were-Rabbit']
Thought: I need to search for Nick Park using the search term "Nick Park" instead of just Nick Park. 


In [9]:
RULE_PREFIX = """
The following are some experience you gather on a similar task of question answering using Wikipedia API. Use these as references to help you perform this task:
"""
print(RULE_PREFIX)


The following are some experience you gather on a similar task of question answering using Wikipedia API. Use these as references to help you perform this task:



In [5]:
from discussion_agents.cog.modules.memory.expel import ExpeLInsightMemory

insights = [('If the initial search does not provide information related to the specific question, try to explore different angles or sources to gather relevant information that may lead to the answer. Consider refining the search query with more specific and relevant keywords to narrow down the search results and find the desired information effectively.',
  5),
 ('If unable to find the answer after exploring different search options, consider reaching out to additional sources or experts for assistance in obtaining the required information.',
  2),
 ('When searching for specific information, try to include relevant keywords related to the topic to narrow down the search results and find the desired information effectively.',
  2),
 ('When searching for specific information, consider verifying the credibility of the sources to ensure the accuracy of the information retrieved.',
  2),
 ('When faced with a search query, consider utilizing different search engines or platforms to gather information and find the desired answer effectively.',
  1)]

insights = [{"insight": insight[0], "score": insight[1]} for insight in insights]
memory = ExpeLInsightMemory(insights=insights, max_num_insights=5, leeway=1)

In [13]:
memory.load_memories()['insights']

[{'insight': 'If the initial search does not provide information related to the specific question, try to explore different angles or sources to gather relevant information that may lead to the answer. Consider refining the search query with more specific and relevant keywords to narrow down the search results and find the desired information effectively.',
  'score': 5},
 {'insight': 'If unable to find the answer after exploring different search options, consider reaching out to additional sources or experts for assistance in obtaining the required information.',
  'score': 2},
 {'insight': 'When searching for specific information, try to include relevant keywords related to the topic to narrow down the search results and find the desired information effectively.',
  'score': 2},
 {'insight': 'When searching for specific information, consider verifying the credibility of the sources to ensure the accuracy of the information retrieved.',
  'score': 2},
 {'insight': 'When faced with a s

In [14]:
insights = memory.load_memories()['insights']

c = "".join([f"{i}. {insight['insight']}\n" for i, insight in enumerate(insights, 1)])

In [17]:
print(a)

The following are some experience you gather on a similar task of question answering using Wikipedia API. Use these as references to help you perform this task:
1. If the initial search does not provide information related to the specific question, try to explore different angles or sources to gather relevant information that may lead to the answer. Consider refining the search query with more specific and relevant keywords to narrow down the search results and find the desired information effectively.
2. If unable to find the answer after exploring different search options, consider reaching out to additional sources or experts for assistance in obtaining the required information.
3. When searching for specific information, try to include relevant keywords related to the topic to narrow down the search results and find the desired information effectively.
4. When searching for specific information, consider verifying the credibility of the sources to ensure the accuracy of the informa

In [18]:
print(c)

1. If the initial search does not provide information related to the specific question, try to explore different angles or sources to gather relevant information that may lead to the answer. Consider refining the search query with more specific and relevant keywords to narrow down the search results and find the desired information effectively.
2. If unable to find the answer after exploring different search options, consider reaching out to additional sources or experts for assistance in obtaining the required information.
3. When searching for specific information, try to include relevant keywords related to the topic to narrow down the search results and find the desired information effectively.
4. When searching for specific information, consider verifying the credibility of the sources to ensure the accuracy of the information retrieved.
5. When faced with a search query, consider utilizing different search engines or platforms to gather information and find the desired answer eff

In [6]:
insights = memory.load_memories()['insights']

a = "The following are some experience you gather on a similar task of question answering using Wikipedia API. Use these as references to help you perform this task:\n"
for i, insight in enumerate(insights, 1):
    a += f"{i}. {insight['insight']}\n"

In [25]:
b = "\n\n".join(memory_dict + ["(END OF EXAMPLES)"]) + "\n"
b += RULE_PREFIX
c = "".join([f"{i}. {insight['insight']}\n" for i, insight in enumerate([], 1)])
b += c

In [26]:
print(b)

The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? 
Thought: I need to search for the creator of "Wallace and Gromit" and find out what other animation comedy they created that involves animated zoo animals and people talking about their homes. 
Action: Action 1: Search[creator of Wallace and Gromit]
Observation 1: Invalid Action. Valid Actions are Lookup[<topic>] Search[<topic>] and Finish[<answer>].
Thought: I should search for the creator of "Wallace and Gromit" by searching for Nick Park. 
Action: Search[Nick Park]
Observation 2: Could not find [Nick Park]. Similar: ['Nick Park', 'Wallace and Gromit', 'Nick Hornby', 'Aardman Animations', 'Chicken Run', 'A Grand Day Out', 'Harry Gregson-Williams', 'Plasticine', 'Nickelodeon', 'Wallace & Gromit: The Curse of the Were-Rabbit']
Thought: I need to search for Nick Park using the search term "Nick Park" instead of just Nick Park. 


In [2]:
from discussion_agents.cog.agent.react import ReActAgent

agent = ReActAgent(llm=llm)

In [3]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"

out = agent.generate(question=question)

<============================================================>
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.
You have a maximum of 6 steps.

Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an ep

In [2]:
from discussion_agents.cog.agent.reflexion import ReflexionCoTAgent

In [3]:
agggent = ReflexionCoTAgent(self_reflect_llm=llm, action_llm=llm, max_trials=2)

In [4]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"
context = 'VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany. It was founded for broadcast of VIVA Germany as VIVA Media AG in 1993 and has been owned by their original concurrent Viacom, the parent company of MTV, since 2004. Viva channels exist in some European countries; the first spin-offs were launched in Poland and Switzerland in 2000.\n\nA Gesellschaft mit beschränkter Haftung (] , abbreviated GmbH ] and also GesmbH in Austria) is a type of legal entity very common in Germany, Austria, Switzerland (where it is equivalent to a S.à r.l.) and Liechtenstein. In the United States, the equivalent type of entity is the limited liability company (LLC). The name of the GmbH form emphasizes the fact that the owners ("Gesellschafter", also known as members) of the entity are not personally liable for the company\'s debts. "GmbH"s are considered legal persons under German and Austrian law. Other variations include mbH (used when the term "Gesellschaft" is part of the company name itself), and gGmbH ("gemeinnützige" GmbH) for non-profit companies.'


out = agggent.generate(question=question, key=key, strategy="reflexion")

PROMPT:

 
Solve a question answering task by having a Thought, then Finish with your answer. Thought can reason about the current situation. Finish[answer] returns the answer and finishes the task.
Here are some examples:

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: Let's think step by step. Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: Let's think step by step. 

PROMPT:

 
Solve a question answering task by having a Thought, then Finish with your answer. Thought can reason about the current situation. Finish[answer] returns the answer and finishes the task.
Here are some examples:

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: Let's think step by step. Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: Let's think step by step. 

In [33]:
from discussion_agents.cog.prompts.react import REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES

print("\n\n".join([REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES] + ["adasd"]) + "\n")



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought 2: It does not mention the eastern sector. So I need to look up eastern sector.
Action 2: Lookup[eastern sector]
Observation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action 3: Search[High Plains]
Observation 3: High Plains refers to one of two distinct land regions:
Thought 4: I need to instead search High Plains (United States).
Action 4: Search[

In [7]:
print(a)

The following are some experience you gather on a similar task of question answering using Wikipedia API. Use these as references to help you perform this task:
1. If the initial search does not provide information related to the specific question, try to explore different angles or sources to gather relevant information that may lead to the answer. Consider refining the search query with more specific and relevant keywords to narrow down the search results and find the desired information effectively.
2. If unable to find the answer after exploring different search options, consider reaching out to additional sources or experts for assistance in obtaining the required information.
3. When searching for specific information, try to include relevant keywords related to the topic to narrow down the search results and find the desired information effectively.
4. When searching for specific information, consider verifying the credibility of the sources to ensure the accuracy of the informa

In [17]:
EXPEL_REFLEXION_REACT_INSTRUCTION = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.
You have a maximum of {max_steps} steps.

Here are some examples:
{examples}

{reflections}

Question: {question}{scratchpad}
"""

examples = "\n\n".join(memory_dict) + "\n\n" + "(END OF EXAMPLES)" + "\n\n"
print(examples)

The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? 
Thought: I need to search for the creator of "Wallace and Gromit" and find out what other animation comedy they created that involves animated zoo animals and people talking about their homes. 
Action: Action 1: Search[creator of Wallace and Gromit]
Observation 1: Invalid Action. Valid Actions are Lookup[<topic>] Search[<topic>] and Finish[<answer>].
Thought: I should search for the creator of "Wallace and Gromit" by searching for Nick Park. 
Action: Search[Nick Park]
Observation 2: Could not find [Nick Park]. Similar: ['Nick Park', 'Wallace and Gromit', 'Nick Hornby', 'Aardman Animations', 'Chicken Run', 'A Grand Day Out', 'Harry Gregson-Williams', 'Plasticine', 'Nickelodeon', 'Wallace & Gromit: The Curse of the Were-Rabbit']
Thought: I need to search for Nick Park using the search term "Nick Park" instead of just Nick Park. 


In [ ]:
agent = ExpeLAgent(
    llm=llm,
    self_reflect_llm=llm,
    action_llm=llm
)

In [ ]:
from discussion_agents.cog.prompts.react import REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES

In [ ]:
a == REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES + "\n(END OF EXAMPLES)\n"

In [ ]:
a = """
Here are some examples:
{examples}
(END OF EXAMPLES)
"""

b = f"""
Here are some examples:
{REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES}
""" + \
"(END OF EXAMPLES)\n"

a.format(examples=REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES) == b

In [ ]:
a

In [ ]:
from discussion_agents.cog.agent.reflexion import ReflexionReActAgent

reflexion_agent = ReflexionReActAgent(
    self_reflect_llm=llm,
    action_llm=llm
)

In [ ]:
reflexion_agent.generate(
    hotpotqa.iloc[0].question,
    hotpotqa.iloc[0].answer
)

In [ ]:
agent.insight_memory.insights

In [ ]:
agent.gather_experience(
    questions=hotpotqa.question.values[:1],
    keys=hotpotqa.answer.values[:1]
)

In [ ]:
agent.insight_memory.insights

In [ ]:
agent.experience_memory.experiences['trajectories'][0]

In [ ]:
agent.experience_memory.success_traj_docs

In [ ]:
agent.experience_memory.vectorstore

In [ ]:
agent.insight_memory.insights